# 🦷 Dental X-Ray Detection Model Training
## Using YOLOv8 on Google Colab (FREE)

This notebook trains an object detection model for panoramic dental X-rays.

## 1️⃣ Setup Environment

In [ ]:
# Check if GPU is available
!nvidia-smi

In [ ]:
# Install required packages
!pip install ultralytics roboflow -q
print("✅ Installation complete!")

## 2️⃣ Download Dataset from Roboflow

In [ ]:
from roboflow import Roboflow
import os

# Initialize Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")  # Replace with your API key

# Download your dataset
print("📥 Downloading dataset...")
project = rf.workspace("dental-yvybz").project("panoramic-xray-mk1hj-fh78n")
dataset = project.version(1).download("yolov8")

print(f"✅ Dataset downloaded to: {dataset.location}")

## 3️⃣ Train the Model

In [ ]:
from ultralytics import YOLO

# Initialize YOLOv8 model
model = YOLO('yolov8n.pt')  # nano model - fast training
# For better accuracy, use: YOLO('yolov8m.pt') or YOLO('yolov8l.pt')

# Train the model
print("🚀 Starting training...")
results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=50,  # Increase for better accuracy (100-200)
    imgsz=640,
    batch=16,
    name='dental_xray_detector',
    patience=10,  # Early stopping
    save=True,
    plots=True
)

print("✅ Training complete!")

## 4️⃣ Evaluate the Model

In [ ]:
# Validate the trained model
metrics = model.val()

print(f"\n📊 Model Performance:")
print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall: {metrics.box.mr:.3f}")

## 5️⃣ Test on Sample Images

In [ ]:
from IPython.display import Image, display
import glob

# Get test images
test_images = glob.glob(f"{dataset.location}/test/images/*.jpg")[:5]

print(f"🔍 Testing on {len(test_images)} sample images...\n")

for img_path in test_images:
    # Run inference
    results = model.predict(img_path, conf=0.4)
    
    # Display result
    result_img = results[0].plot()
    
    # Save and display
    save_path = f"result_{os.path.basename(img_path)}"
    results[0].save(save_path)
    
    print(f"Image: {os.path.basename(img_path)}")
    print(f"Detections: {len(results[0].boxes)}")
    display(Image(filename=save_path))
    print("\n" + "="*50 + "\n")

## 6️⃣ Export Model Weights

In [ ]:
import shutil

# Find the best model weights
best_model_path = "runs/detect/dental_xray_detector/weights/best.pt"

print(f"📦 Best model saved at: {best_model_path}")
print("\n💾 To download:")
print("1. Click the folder icon on the left")
print("2. Navigate to runs/detect/dental_xray_detector/weights/")
print("3. Right-click 'best.pt' and select 'Download'")

# Create a zip file for easy download
!zip -r dental_xray_model.zip runs/detect/dental_xray_detector/
print("\n✅ Created dental_xray_model.zip with all training results")

## 7️⃣ Export for Deployment (Optional)

In [ ]:
# Export to different formats
print("🔄 Exporting to ONNX format...")
model.export(format='onnx')

print("🔄 Exporting to TorchScript format...")
model.export(format='torchscript')

print("✅ Export complete!")

## 8️⃣ Download to Google Drive (Optional)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy model to Google Drive
!cp -r runs/detect/dental_xray_detector /content/drive/MyDrive/
!cp dental_xray_model.zip /content/drive/MyDrive/

print("✅ Model saved to Google Drive!")